In [4]:
using DelimitedFiles
using Graphs

In [5]:
import Pkg; Pkg.add("Graphs")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [6]:
struct Instance
    graph::SimpleDiGraph
    vertex_weight::Vector{Float64}
    edge_weight::Matrix{Float64}
    pairs::Vector{Int}
    altruists::Vector{Int}
    nb_pairs::Int
    nb_altruists::Int
    max_cycle_length::Int
    max_chain_length::Int
    is_vertex_weighted::Bool
    
    function Instance(g::SimpleDiGraph, vertex_weight::Array{Float64}, K::Int, L::Int = 0)
        P = [v for v in vertices(g) if vertex_weight[v] != 0.0]
        A = [v for v in vertices(g) if vertex_weight[v] == 0.0]
        edge_weight = zeros(nv(g), nv(g))
        for e in edges(g)
            edge_weight[e.src,e.dst] = vertex_weight[e.dst]
        end

        return new(g, vertex_weight, edge_weight, P, A, length(P), length(A), K, L, true)
    end
           
    function Instance(filename::String, K::Int, L::Int = 0)
        inst = string(filename)
        data_folder = "data" # joinpath(pkgdir(KidneyExchange), "data")
        wmd_file = joinpath(data_folder, join([inst, ".wmd"]))
        dat_file = joinpath(data_folder, join([inst, ".dat"]))

        g, edge_weight, is_altruist = read_kep_file(wmd_file, dat_file)
        P = [v for v in vertices(g) if !is_altruist[v]]
        A = [v for v in vertices(g) if is_altruist[v]]
        vertex_weight = zeros(nv(g))
        is_vertex_weighted = true
        for v in P
            vertex_weight[v] = 1.0
            if indegree(g, v) >= 1
                vertex_weight[v] = edge_weight[inneighbors(g,v)[1],v]
                for u in inneighbors(g,v)
                    if edge_weight[u,v] != vertex_weight[v]
                        is_vertex_weighted = false
                        break
                    end
                end
                if !is_vertex_weighted  break   end
            end
        end
        if !is_vertex_weighted
            println("the instance is not vertex weighted!")
        end
        return new(g, vertex_weight, edge_weight, P, A, length(P), length(A), K, L, is_vertex_weighted)
    end 
                            
end

In [7]:
function read_kep_file(wmd_file::AbstractString, dat_file::AbstractString)

    #wmd_file_name = split(split(wmd_file, '/')[end], '.')[1]
    #dat_file_name = split(split(dat_file, '/')[end], '.')[1]

	wmd_file_name = first(splitext(last(splitpath(wmd_file))))
	dat_file_name = first(splitext(last(splitpath(dat_file))))

    wmd_file_name == dat_file_name || throw(ArgumentError(".wmd and .dat files do not correspond to the same dataset."))
	isfile(abspath(wmd_file)) || throw(ArgumentError("$(abspath(wmd_file)): file not found."))
	isfile(abspath(dat_file)) || throw(ArgumentError(".dat file not found."))

	# Get the number of vertices and edges from the first line of wmd file
    wmd_io = open(wmd_file, "r")
	splitted_line = split(readline(wmd_io), ',')
	nb_vertices = parse(Int, splitted_line[1])
	nb_edges = parse(Int, splitted_line[2])

    # Extract meta information from the .dat file
    file = readdlm(dat_file, '\n')
	is_altruist = falses(nb_vertices)
	ind = 1
    for line in file[2:end]
        splitted_line = split(line, ',')
        if Bool(parse(Int, splitted_line[7]))
			is_altruist[ind] = true
		end
		ind += 1
    end

    # Extract the graph structure from the .wmd file
    # skip next nb_vertices lines, which are redundant with the data contained in the .dat file
    for i in 1:nb_vertices
        readline(wmd_io)
    end

    # read the set of edges
	in_list = Vector{Vector{Int}}(undef, nb_vertices)
	out_list = Vector{Vector{Int}}(undef, nb_vertices)
	for u in 1:nb_vertices
		in_list[u] = Vector{Int}()
		out_list[u] = Vector{Int}()
	end
	ne = 0
	edge_weight = zeros(nb_vertices, nb_vertices)
    while !eof(wmd_io)
        splitted_line = split(readline(wmd_io), ',')
        # /!\ Pairs are numbered from 0 in the second part of the file
        src = parse(Int, splitted_line[1]) + 1
        dst = parse(Int, splitted_line[2]) + 1
        weight = parse(Float64, splitted_line[3])

        # do not add an edge that has an altruist as destination or that has a zero weight
		if !is_altruist[dst] && weight > 0.0
			ne += 1
			push!(out_list[src], dst)
			push!(in_list[dst], src)
			edge_weight[src, dst] = weight
		end
    end
	for u in 1:nb_vertices
		sort!(out_list[u])
		sort!(in_list[u])
	end
    return SimpleDiGraph(ne, out_list, in_list), edge_weight, is_altruist
end

read_kep_file (generic function with 1 method)

In [8]:
data_folder = string(@__DIR__,"/data")
instance_name = "MD-00001-00000010"
filename = joinpath(data_folder, join([instance_name]))
new_inst = Instance(filename, 3, 0)

Instance({16, 47} directed simple Int64 graph, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], Int64[], 16, 0, 3, 0, true)

In [9]:
"The instance is represented by the simple directed graph: $(new_inst.graph)"

"The instance is represented by the simple directed graph: {16, 47} directed simple Int64 graph"

In [10]:
"Vertex weights are given as $(new_inst.vertex_weight)"

"Vertex weights are given as [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"

In [11]:
new_inst.edge_weight[1,2]

0.0

In [41]:
new_inst.pairs

16-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16

In [42]:
new_inst.altruists

0-element Array{Int64,1}

In [43]:
new_inst.is_vertex_weighted

true

In [24]:
"This graph has $(nv(new_inst.graph)) nodes."

"This graph has 16 nodes"

In [25]:
"This graph has $(ne(new_inst.graph)) arcs."

"This graph has 47 arcs"

In [31]:
"The set of nodes is given by $(vertices(new_inst.graph)). This is an iterable object containing all vertices."

"The set of nodes is given by Base.OneTo(16). This is an iterable object containing all arcs."

In [32]:
#for v in vertices(new_inst.graph)
#    println(v)
#end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [28]:
"The set of arcs is given by $(edges(new_inst.graph)). This is an iterable object containing all arcs."

"The set of arcs is given by SimpleEdgeIter 47"

In [33]:
#for e in edges(new_inst.graph)
#    println("$e has $(src(e)) as its origin and $(dst(e)) as its destination")
#end

Edge 1 => 3 has 1 as its origin and 3 as its destination
Edge 1 => 16 has 1 as its origin and 16 as its destination
Edge 2 => 4 has 2 as its origin and 4 as its destination
Edge 2 => 14 has 2 as its origin and 14 as its destination
Edge 3 => 4 has 3 as its origin and 4 as its destination
Edge 3 => 14 has 3 as its origin and 14 as its destination
Edge 4 => 6 has 4 as its origin and 6 as its destination
Edge 4 => 13 has 4 as its origin and 13 as its destination
Edge 4 => 16 has 4 as its origin and 16 as its destination
Edge 5 => 1 has 5 as its origin and 1 as its destination
Edge 5 => 2 has 5 as its origin and 2 as its destination
Edge 5 => 4 has 5 as its origin and 4 as its destination
Edge 5 => 6 has 5 as its origin and 6 as its destination
Edge 5 => 8 has 5 as its origin and 8 as its destination
Edge 5 => 10 has 5 as its origin and 10 as its destination
Edge 5 => 11 has 5 as its origin and 11 as its destination
Edge 5 => 13 has 5 as its origin and 13 as its destination
Edge 5 => 14 ha